<a href="https://colab.research.google.com/github/solomonmelwin/solomonmelwin/blob/main/weather_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [ ]:
!pip install kaggle
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!kaggle datasets download nikhil7280/weather-type-classification
!unzip weather-type-classification.zip


Dataset URL: https://www.kaggle.com/datasets/nikhil7280/weather-type-classification
License(s): other
weather-type-classification.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  weather-type-classification.zip
replace weather_classification_data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:

import pandas as pd
print(df.head())df = pd.read_csv('weather_classification_data.csv')


   Temperature  Humidity  Wind Speed  Precipitation (%)    Cloud Cover  \
0         14.0        73         9.5               82.0  partly cloudy   
1         39.0        96         8.5               71.0  partly cloudy   
2         30.0        64         7.0               16.0          clear   
3         38.0        83         1.5               82.0          clear   
4         27.0        74        17.0               66.0       overcast   

   Atmospheric Pressure  UV Index  Season  Visibility (km)  Location  \
0               1010.82         2  Winter              3.5    inland   
1               1011.43         7  Spring             10.0    inland   
2               1018.72         5  Spring              5.5  mountain   
3               1026.25         7  Spring              1.0   coastal   
4                990.67         1  Winter              2.5  mountain   

  Weather Type  
0        Rainy  
1       Cloudy  
2        Sunny  
3        Sunny  
4        Rainy  


In [ ]:
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13200 entries, 0 to 13199
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Temperature           13200 non-null  float64
 1   Humidity              13200 non-null  int64  
 2   Wind Speed            13200 non-null  float64
 3   Precipitation (%)     13200 non-null  float64
 4   Cloud Cover           13200 non-null  object 
 5   Atmospheric Pressure  13200 non-null  float64
 6   UV Index              13200 non-null  int64  
 7   Season                13200 non-null  object 
 8   Visibility (km)       13200 non-null  float64
 9   Location              13200 non-null  object 
 10  Weather Type          13200 non-null  object 
dtypes: float64(5), int64(2), object(4)
memory usage: 1.1+ MB
None


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Assuming there is a column named 'weather_type' as the target label
label_col = 'Weather Type'  # Update this based on the actual column name in the dataset
X = df.drop(columns=[label_col])  # Features
y = df[label_col]  # Target label

# Encode the target labels
le = LabelEncoder()
y = le.fit_transform(y)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Separate numerical and categorical features
numerical_features = X_train.select_dtypes(include=['number']).columns
categorical_features = X_train.select_dtypes(exclude=['number']).columns

# Create transformers for numerical and categorical features
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Fit and transform the data
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# Initialize models
log_reg = LogisticRegression(max_iter=1000)
decision_tree = DecisionTreeClassifier()
random_forest = RandomForestClassifier(n_estimators=100)
knn = KNeighborsClassifier(n_neighbors=5)

# Create a dictionary of models to compare
models = {
    "Logistic Regression": log_reg,
    "Decision Tree": decision_tree,
    "Random Forest": random_forest,
    "KNN": knn
}


In [ ]:
def evaluate_models(models, X_train, y_train, X_test, y_test):
    results = {}
    for name, model in models.items():
        print(f"\nTraining {name}...")
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        print(f"{name} Accuracy: {accuracy:.4f}")
        print(f"Classification Report for {name}:\n{classification_report(y_test, y_pred)}")
        print(f"Confusion Matrix for {name}:\n{confusion_matrix(y_test, y_pred)}")
        results[name] = accuracy
    return results


In [ ]:
results = evaluate_models(models, X_train, y_train, X_test, y_test)



Training Logistic Regression...
Logistic Regression Accuracy: 0.8708
Classification Report for Logistic Regression:
              precision    recall  f1-score   support

           0       0.82      0.84      0.83       651
           1       0.86      0.84      0.85       647
           2       0.89      0.94      0.91       701
           3       0.92      0.86      0.89       641

    accuracy                           0.87      2640
   macro avg       0.87      0.87      0.87      2640
weighted avg       0.87      0.87      0.87      2640

Confusion Matrix for Logistic Regression:
[[549  57  20  25]
 [ 46 543  50   8]
 [ 21   8 658  14]
 [ 52  25  15 549]]

Training Decision Tree...
Decision Tree Accuracy: 0.9098
Classification Report for Decision Tree:
              precision    recall  f1-score   support

           0       0.89      0.89      0.89       651
           1       0.90      0.90      0.90       647
           2       0.93      0.94      0.94       701
           3 

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers

# Define a simple neural network
nn_model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(len(le.classes_), activation='softmax')  # Output layer for multi-class classification
])

# Compile the neural network
nn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the neural network
history = nn_model.fit(X_train, y_train, epochs=20, validation_split=0.2, batch_size=32)

# Evaluate the neural network
nn_loss, nn_accuracy = nn_model.evaluate(X_test, y_test)
print(f"\nNeural Network Accuracy: {nn_accuracy:.4f}")


Epoch 1/20
264/264 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.6516 - loss: 0.9131 - val_accuracy: 0.8939 - val_loss: 0.3269
Epoch 2/20
264/264 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8911 - loss: 0.3215 - val_accuracy: 0.9115 - val_loss: 0.2622
Epoch 3/20
264/264 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9045 - loss: 0.2587 - val_accuracy: 0.9081 - val_loss: 0.2476
Epoch 4/20
264/264 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9093 - loss: 0.2381 - val_accuracy: 0.9115 - val_loss: 0.2329
Epoch 5/20
264/264 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9136 - loss: 0.2317 - val_accuracy: 0.9157 - val_loss: 0.2211
Epoch 6/20
264/264 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9111 - loss: 0.2182 - val_accuracy: 0.9115 - val_loss: 0.2176
Epoch 7/20
264/264 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9175 - loss: 0.2055 - val_accuracy: 0.9138 - val_loss: 0.2157
Epoch 8/20
264/264 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9168 - loss: 0.2027 - val_accuracy: 0.

In [ ]:
# Predictions for the neural network
y_pred_nn = nn_model.predict(X_test).argmax(axis=1)
print(f"Classification Report for Neural Network:\n{classification_report(y_test, y_pred_nn)}")
print(f"Confusion Matrix for Neural Network:\n{confusion_matrix(y_test, y_pred_nn)}")



# Combine the accuracy of all models including neural network
results["Neural Network"] = nn_accuracy



83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Classification Report for Neural Network:
              precision    recall  f1-score   support

           0       0.91      0.87      0.89       651
           1       0.88      0.91      0.89       647
           2       0.93      0.94      0.94       701
           3       0.91      0.90      0.90       641

    accuracy                           0.91      2640
   macro avg       0.91      0.91      0.91      2640
weighted avg       0.91      0.91      0.91      2640

Confusion Matrix for Neural Network:
[[567  38  18  28]
 [ 28 589  15  15]
 [  7  16 662  16]
 [ 21  27  16 577]]


In [ ]:
print("\nModel Accuracies:")
for model_name, acc in results.items():
    print(f"{model_name}: {acc:.4f}")



Model Accuracies:
Logistic Regression: 0.8708
Decision Tree: 0.9098
Random Forest: 0.9140
KNN: 0.8966
Neural Network: 0.9072


In [ ]:
best_model = max(results, key=results.get)
print(f"\nThe best performing model is: {best_model} with accuracy of {results[best_model]:.4f}")



The best performing model is: Random Forest with accuracy of 0.9140


In [ ]:
# Step 1: Choose the 3 features you want to use for retraining
selected_features = ['Temperature', 'Humidity', 'Wind Speed']  # Replace with the actual feature names

# Subset the dataset for these 3 features
X = df[selected_features]
y = df["Weather Type"]  # Target label (e.g., 'weather_type')

# Continue with preprocessing and training the model
le = LabelEncoder()
y = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train Random Forest with the selected features
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, y_train)



RandomForestClassifier()

In [ ]:
import numpy as np

# Function to take user input for 3 features and predict weather type
def predict_weather(model, scaler, le):
    print("Enter the values for the following 3 features:")

    # 3 features used in prediction (replace with actual feature names from your dataset)
    temperature = float(input("Temperature (in Celsius): "))
    humidity = float(input("Humidity (percentage): "))
    wind_speed = float(input("Wind Speed (km/h): "))

    # Create an array of input values for only the selected 3 features
    user_input = np.array([[temperature, humidity, wind_speed]])

    # Scale the input using the scaler fitted on the training data
    user_input_scaled = scaler.transform(user_input)

    # Predict the weather type using the trained Random Forest model
    prediction = model.predict(user_input_scaled)

    # Decode the predicted label to get the weather type
    predicted_weather_type = le.inverse_transform(prediction)

    print(f"The predicted weather type is: {predicted_weather_type[0]}")

# Call this function to predict based on user input
predict_weather(random_forest, scaler, le)


Enter the values for the following 3 features:
Temperature (in Celsius): 30
Humidity (percentage): 76
Wind Speed (km/h): 13
The predicted weather type is: Cloudy


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
